In [44]:
import sys
sys.path.append('../') 
from imports import *
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import ParameterGrid


In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [20]:
# Load the saved datasets
train_data = np.load('./fashion-mnist-processed-data/train_data.npy')
val_data = np.load('./fashion-mnist-processed-data/val_data.npy')
test_data = np.load('./fashion-mnist-processed-data/test_data.npy')

train_targets = np.load('./fashion-mnist-processed-data/train_targets.npy')
val_targets = np.load('./fashion-mnist-processed-data/val_targets.npy')
test_targets = np.load('./fashion-mnist-processed-data/test_targets.npy')

In [21]:
train_data = np.expand_dims(train_data, axis=1)
val_data = np.expand_dims(val_data, axis=1)
test_data = np.expand_dims(test_data, axis=1)


In [22]:
# Converting to tensors
train_data, train_targets = torch.tensor(train_data, dtype=torch.float32), torch.tensor(train_targets, dtype=torch.long)
val_data, val_targets = torch.tensor(val_data, dtype=torch.float32), torch.tensor(val_targets, dtype=torch.long)
test_data, test_targets = torch.tensor(test_data, dtype=torch.float32), torch.tensor(test_targets, dtype=torch.long)


In [23]:
# Normalizeing the data
train_data = train_data/ 255.0
val_data = val_data/255.0
test_data = test_data/255.0

In [24]:
# Data loaders and set batch size for training, validation, testing
batch_size = 64
train_loader = DataLoader(TensorDataset(train_data, train_targets), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(val_data, val_targets), batch_size=batch_size, shuffle=False)
test_loader = DataLoader(TensorDataset(test_data, test_targets), batch_size=batch_size, shuffle=False)


In [25]:
#Single layer CNN model
class SingleLayerCNN(nn.Module):
    def __init__(self):
        super(SingleLayerCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(32 * 28 * 28, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


In [26]:

model = SingleLayerCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [28]:
# Training and validation for certain epochs
epochs = 10
for epoch in range(epochs):
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation phase
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        total_train = 0
        correct_train = 0
    
        for images_train, labels_train in train_loader:
            outputs_train = model(images_train)
            _, predicted_train = torch.max(outputs_train.data, 1)
            total_train += labels_train.size(0)
            correct_train += (predicted_train == labels_train).sum().item()
        for images, labels in val_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
        train_accuracy = 100 * correct_train / total_train
        val_accuracy = 100 * correct / total
        print(f'Epoch [{epoch+1}/{epochs}], Training accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%')


Epoch [1/10], Training accuracy: 96.73%, Validation Accuracy: 90.42%
Epoch [2/10], Training accuracy: 96.41%, Validation Accuracy: 90.52%
Epoch [3/10], Training accuracy: 96.97%, Validation Accuracy: 90.42%
Epoch [4/10], Training accuracy: 96.91%, Validation Accuracy: 89.73%
Epoch [5/10], Training accuracy: 97.16%, Validation Accuracy: 90.22%
Epoch [6/10], Training accuracy: 97.51%, Validation Accuracy: 90.35%
Epoch [7/10], Training accuracy: 98.22%, Validation Accuracy: 90.43%
Epoch [8/10], Training accuracy: 97.92%, Validation Accuracy: 90.45%
Epoch [9/10], Training accuracy: 98.30%, Validation Accuracy: 90.25%
Epoch [10/10], Training accuracy: 98.30%, Validation Accuracy: 90.18%


In [29]:
# Prediction on a single item from test set
model.eval()
with torch.no_grad():
    images, labels = next(iter(test_loader))
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    print(f'Predicted label: {predicted[0].item()}, Actual label: {labels[0].item()}')

Predicted label: 9, Actual label: 9


In [30]:
# Evaluation on test data
model.eval()
with torch.no_grad():
    correct_test = 0
    total_test = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

    test_accuracy = 100 * correct_test / total_test
    print(f'Test Accuracy: {test_accuracy:.2f}%')


Test Accuracy: 90.55%


In [32]:
# Now let us perform hyperparamter tuning to find optimal parameters and replace them where needed!
# Defining the parametr grid for hyperparameter tuning
param_grid = {
    'lr': [0.001, 0.01],
    'batch_size': [32, 64, 128],
    'epochs': [50, 100]
}

In [36]:
# Function to train and evaluate the model
def train_val_evaluate(model, train_loader, val_loader, criterion, optimizer, epochs=3):
    
    for epoch in range(epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    model.eval()
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_accuracy = 100 * correct_val / total_val
    return val_accuracy

In [37]:
# Let us begin Hyperparameter tuning
best_accuracy = 0
best_params = None

for params in ParameterGrid(param_grid):
    # Update batch size for data loaders
    train_loader = DataLoader(TensorDataset(train_data, train_targets), batch_size=params['batch_size'], shuffle=True)
    val_loader = DataLoader(TensorDataset(val_data, val_targets), batch_size=params['batch_size'], shuffle=False)
    model = SingleLayerCNN()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=params['lr'])

    # Training and evaluating the model
    accuracy = train_val_evaluate(model, train_loader, val_loader, criterion, optimizer, params['epochs'])
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = params

    print(f"Params: {params}, Validation Accuracy: {accuracy:.2f}%")

print(f"Best Params: {best_params}, Best Accuracy: {best_accuracy:.2f}%")

Params: {'batch_size': 32, 'lr': 0.001}, Validation Accuracy: 89.78%
Params: {'batch_size': 32, 'lr': 0.01}, Validation Accuracy: 88.42%
Params: {'batch_size': 32, 'lr': 0.1}, Validation Accuracy: 55.72%
Params: {'batch_size': 64, 'lr': 0.001}, Validation Accuracy: 90.00%
Params: {'batch_size': 64, 'lr': 0.01}, Validation Accuracy: 87.30%
Params: {'batch_size': 64, 'lr': 0.1}, Validation Accuracy: 81.40%
Params: {'batch_size': 128, 'lr': 0.001}, Validation Accuracy: 89.40%
Params: {'batch_size': 128, 'lr': 0.01}, Validation Accuracy: 89.12%
Params: {'batch_size': 128, 'lr': 0.1}, Validation Accuracy: 10.00%
Best Params: {'batch_size': 64, 'lr': 0.001}, Best Accuracy: 90.00%
